# 6 - Prise en compte des données de température pour affiner le modèle

Afin d'améliorer la qualité de la prédiction, il est possible de passer d'un modèle ARMA à un modèle ARMAX pour la consommation désaisonnalisée. 

Un processus ARMAX(p,q) est de la forme $Y_t= \sum_{i=1}^{4} \phi_i Y_{t-i}+ \sum_{j=1}^{2} \theta_j\varepsilon_{t-j}+ \beta^\top X_t+ \varepsilon_t$ avec $(\varepsilon_t)$ un bruit blanc.

$X_t$ contient des variables exogènes, on prendra $(X_t)=(T_{t-2},T_{t-1},T_{t})^T$ car il est raisonnable de supposer dans un premier temps que les variations instantanées de consommation électrique à l'échelle de la ville de Paris dépendent essentiellement des variations de température survenues dans les deux heures précédentes. 